# 11. 그룹연산

- groupby method
- 분할 - 반영 - 결합 (Split-Apply-Combine)
- SQL의 GROUPBY 구문과 비슷

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 11.1. 데이터 집계

In [38]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kuggle/data/data/gapminder.tsv', sep='\t')
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [3]:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


### 11.1.1. 분할-반영-결합 과정

In [4]:
years = df.year.unique()
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [5]:
# 분할
y1952 = df.loc[df.year == 1952, :]
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [6]:
# 반영
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)

49.057619718309866


In [7]:
# 반영 반복

year_means = []

for y in years:
  year_means.append(df.loc[df.year == y, :].lifeExp.mean())

print(year_means)

[49.057619718309866, 51.50740112676056, 53.609249014084504, 55.678289577464795, 57.64738647887324, 59.57015746478874, 61.53319718309859, 63.21261267605633, 64.16033802816901, 65.01467605633802, 65.69492253521126, 67.00742253521126]


In [8]:
df2 = pd.DataFrame({'year': years,
                    'lifeExp': year_means})
print(df2)

    year    lifeExp
0   1952  49.057620
1   1957  51.507401
2   1962  53.609249
3   1967  55.678290
4   1972  57.647386
5   1977  59.570157
6   1982  61.533197
7   1987  63.212613
8   1992  64.160338
9   1997  65.014676
10  2002  65.694923
11  2007  67.007423


### 11.1.2. groupby method와 함께 사용하는 집계 메서드

| 메서드              | 설명                                                        |
|---------------------|-------------------------------------------------------------|
| count               | 누락값을 제외한 데이터 수를 반환                            |
| size                | 누락값을 포함한 데이터 수를 반환                            |
| mean                | 평균값 반환                                                 |
| std                 | 표준편차 반환                                               |
| min                 | 최소값 반환                                                 |
| quantile(q=0.25)    | 백분위수 25%                                                |
| quantile(q=0.50)    | 백분위수 50%                                                |
| quantile(q=0.75)    | 백분위수 75%                                                |
| max                 | 최댓값 반환                                                 |
| sum                 | 전체 합 반환                                                |
| var                 | 분산 반환                                                   |
| sem                 | 평균의 표준편차 반환                                        |
| describe            | 데이터 수, 평균, 표준편차, 최소값, 백분위수(25, 50, 75%), 최댓값을 모두 반환 |
| first               | 첫 번째 행 반환                                             |
| last                | 마지막 행 반환                                              |
| nth                 | n번째 행 반환                                               |


### 11.1.3. agg method로, 사용자 함수와 groupby method 조합하기

In [9]:
def my_mean(values):
  n = len(values)
  sum = 0
  for value in values:
    sum += value
  return sum / n

In [10]:
# agg method를 통해서 사용자 함수를 적용함.
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [11]:
def my_mean_diff(values, diff_value):
  n = len(values)
  sum = 0
  for value in values:
    sum += value
  mean = sum / n
  return mean - diff_value

In [12]:
global_mean = df.lifeExp.mean()
print(global_mean)

59.474439366197174


In [13]:
agg_mean_diff= df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean) #두 번째 인자 전달
print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


### 11.1.4.여러 개의 집계 메서드 한 번에 사용하기

In [14]:
# agg method를 통해 여러 method를 함번에 사용
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

      count_nonzero       mean        std
year                                     
1952            142  49.057620  12.225956
1957            142  51.507401  12.231286
1962            142  53.609249  12.097245
1967            142  55.678290  11.718858
1972            142  57.647386  11.381953
1977            142  59.570157  11.227229
1982            142  61.533197  10.770618
1987            142  63.212613  10.556285
1992            142  64.160338  11.227380
1997            142  65.014676  11.559439
2002            142  65.694923  12.279823
2007            142  67.007423  12.073021


<ipython-input-14-4033fe69f648>:3: FutureWarning: The provided callable <function mean at 0x7ec8d1303d90> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
<ipython-input-14-4033fe69f648>:3: FutureWarning: The provided callable <function std at 0x7ec8d1303eb0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])


In [15]:
#  Pandas의 미래 버전에서 함수 호출 방식이 변경될 수 있다는 것을 알리는 것으로, 현재의 동작을 유지하고 싶다면 함수 대신 해당 동작을 수행하는 문자열을 사용
gdf_new = df.groupby('year').lifeExp.agg([np.count_nonzero, "mean", "std"])
print(gdf_new)

      count_nonzero       mean        std
year                                     
1952            142  49.057620  12.225956
1957            142  51.507401  12.231286
1962            142  53.609249  12.097245
1967            142  55.678290  11.718858
1972            142  57.647386  11.381953
1977            142  59.570157  11.227229
1982            142  61.533197  10.770618
1987            142  63.212613  10.556285
1992            142  64.160338  11.227380
1997            142  65.014676  11.559439
2002            142  65.694923  12.279823
2007            142  67.007423  12.073021


In [16]:
# dictionary를 사용하여 column name을 설정하고 사용할 method 이름을 작성
gdf_dict = df.groupby('year').agg({'lifeExp': 'mean', 'pop': 'median', 'gdpPercap': 'median'})
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371108


## 11.2. 데이터 변환

- transform method는 데이터와 method를 일대일로 대응시켜 계산
- 말그대로 데이터를 변환

### 11.2.1. 표준 점수 계산하기

In [17]:
def my_zscore(x):
  return (x - x.mean()) / x.std()

transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [18]:
print(df.shape)
print(transform_z.shape)

(1704, 6)
(1704,)


### 11.2.2. 누락값을 평균값으로

In [19]:
import seaborn as sns
import numpy as np

np.random.seed(42) # 교재의 예시와 동일하게 결과를 출력하기 위해 seed를 설정함.

tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN

print(tips_10)

# np.random 모듈
# permutation : 주어진 배열이나 범위의 순서를 무작위로 섞은 새로운 배열을 반환
# index : Pandas 데이터프레임 tips_10의 인덱스를 반환

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [33]:
# 여성과 남성의 누락값 수가 다름.
count_sex = tips_10.groupby('sex', observed=False).count()
count_sex_true = tips_10.groupby('sex', observed=True).count()
print(count_sex)
print(count_sex_true)

# 경고의 원인 : groupby method의 observed 인자에 대한 기본값이 변경될 예정
# 현재는 observed=False가 기본값이지만, 이후 버전에서는 observed=True로 변경
# observed parameter : 범주형 변수를 사용하는 그룹화에서 사용. False로 설정되면 모든 범주의 조합이 출력됩니다. 반면 True로 설정되면 실제로 관찰된 (데이터에서 존재하는) 조합만 출력됩니다.

        total_bill  tip  smoker  day  time  size
sex                                             
Male             7    7       7    7     7     7
Female           3    3       3    3     3     3
        total_bill  tip  smoker  day  time  size
sex                                             
Male             7    7       7    7     7     7
Female           3    3       3    3     3     3


In [23]:
# 성별(x)에 따라 평균으로 채우는 함수
def fill_na_mean(x):
  avg = x.mean()
  return x.fillna(avg)

In [24]:
# 성별로 구분한 total_bill열의 데이터를 fill_na_mean 함수에 전달
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)


# 새로운 열 추가
tips_10['fill_total_bill'] = total_bill_group_mean

print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


<ipython-input-24-009bb7e9262d>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)


## 11.3. 데이터 필터링

In [25]:
tips = sns.load_dataset('tips')
print(tips.shape)
print(tips.head())

(244, 7)
   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [26]:
print(tips['size'].value_counts())
# Return a Series containing the frequency of each distinct row in the Dataframe.
# count()와 다름 주의

size
2    156
3     38
4     37
5      5
1      4
6      4
Name: count, dtype: int64


In [28]:
# 각 size 별로 30번 이상의 주문 건이 있는 테이블 그룹화
tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count() >=30)

print(tips_filtered.shape)
print()
print(tips_filtered['size'].value_counts())

(231, 7)

size
2    156
3     38
4     37
Name: count, dtype: int64


## 11.4. 그룹 오브젝트

-  groupby method가 반환하는 값

In [29]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [30]:
grouped = tips_10.groupby('sex')
print(grouped)

<ipython-input-30-e73486985672>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = tips_10.groupby('sex')


In [34]:
# group object의 group attribute 출력 - group화한 index 확인
print(grouped.groups)

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


### 11.4.1. 한 번에 그룹 오브젝트 계산하기

- 범주형/연속형을 자동으로 인식하고 집계 method 적용 - 안됨.

In [39]:
avgs = grouped.agg({'total_bill':'mean', 'tip':'mean', 'size':'mean'})
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


### 11.4.2. 그룹 오브젝트 활용하기

In [40]:
# get_group : 특정 데이터 추출
female = grouped.get_group('Female')
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [41]:
for sex_group in grouped:
  print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [42]:
for sex_group in grouped:
  print('the type is: {}\n'.format(type(sex_group)))
  print('the length is: {}\n'.format(len(sex_group)))

  first_element = sex_group[0]
  print('the first element is: {}\n'.format(first_element))
  print('it has a type of: {}\n'.format(type(sex_group[0])))

  second_element = sex_group[1]
  print('the second element is: {}\n'.format(second_element))
  print('it has a type of: {}\n'.format(type(second_element)))

  print('what we have:')
  print(sex_group)

  break


the type is: <class 'tuple'>

the length is: 2

the first element is: Male

it has a type of: <class 'str'>

the second element is:      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2

it has a type of: <class 'pandas.core.frame.DataFrame'>

what we have:
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  

### 11.4.3. 여러 열을 사용해 그룹 오브젝트 만드록 계산

In [48]:
bill_sex_time = tips_10.groupby(['sex', 'time'], observed=False)
group_avg = bill_sex_time[['total_bill', 'tip', 'size']].mean()

print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [49]:
print(type(group_avg))
print()
print(group_avg.columns)

<class 'pandas.core.frame.DataFrame'>

Index(['total_bill', 'tip', 'size'], dtype='object')


In [50]:
print(group_avg.index)

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [52]:
grouped_method = tips_10.groupby(['sex', 'time'])[['total_bill', 'tip', 'size']].mean().reset_index()
print(grouped_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


<ipython-input-52-57f8eb5a4031>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_method = tips_10.groupby(['sex', 'time'])[['total_bill', 'tip', 'size']].mean().reset_index()
